In [ ]:
%reload_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv
%matplotlib inline

In [ ]:
from xcube_sh.config import CubeConfig
from xcube_sh.cube import open_cube
from xcube_sh.sentinelhub import SentinelHub
import xarray as xr
import numpy as np
import os
import sys
import json
import shapely.geometry
import IPython.display
import matplotlib.pyplot as plt

In [ ]:
sys.path.insert(0,os.path.dirname('../src/'))
from GIS_utils import bbox_from_point
from preprocess import generate_bg_from_s1, remove_s1_empty_nans, generate_landwater_mask

In [ ]:
SH = SentinelHub()
print(SH.dataset_names)

In [ ]:
SH.band_names('S1GRD')

In [ ]:
data_dir = "/home/jovyan/data" # data directory (path)
RADIUS = 500 # AOI radius in meters

spatial_res = 0.00018
start_date = '2019-01-01'
end_date = '2019-06-01'

with open(os.path.join(data_dir, 'aoi.json'), 'r') as f:
    aoi_file = json.load(f)
    coord = aoi_file['Final']['Venice'][0]
    lat, lon = coord[0], coord[1]
    print('{}, {}'.format(lat, lon))

In [ ]:
# location purely on water
# lon = 28.994
# lat = 41.021
bbox = bbox_from_point(lat=lat, lon=lon, r=RADIUS) # WGS84 coordinates

In [ ]:
bbox = bbox_from_point(lat=lat, lon=lon, r=RADIUS) # WGS84 coordinates
IPython.display.GeoJSON(shapely.geometry.box(*bbox).__geo_interface__)

In [ ]:
cube_config = CubeConfig(dataset_name='S1GRD',
                         band_names= ["HH", "HV", "VH", "VV", "HH+HV", "VV+VH"], ## HH,'HV' , 'HH+HV' gives NaN/Zero Exception HH+HV, VV+VH, HH, VV
                         crs = "http://www.opengis.net/def/crs/EPSG/0/4326",
                         spatial_res = 0.00018,
                         geometry=bbox,
                         time_range=[start_date, end_date],
                         time_period='1D')

In [ ]:
cube = open_cube(cube_config)
cube

In [ ]:
cube = remove_s1_empty_nans(cube)
cube

In [ ]:
plt.figure(figsize=(15, 15))
plt.subplot(2, 2, 1)
plt.hist(np.squeeze(cube.VH.values.reshape(1, -1)), bins=np.arange(0, 1, 0.01))
plt.title("VH")
plt.xticks(np.arange(0, 1, 0.1))
plt.subplot(2, 2, 2)
plt.hist(np.squeeze(cube.VV.values.reshape(1, -1)), bins=np.arange(0, 1, 0.01))
plt.title("VV")
plt.xticks(np.arange(0, 1, 0.1))


In [ ]:
cube.VV.plot.imshow(col='time', col_wrap=4, vmax=0.15) # 0.15 for VV

In [ ]:
cube.VH.plot.imshow(col='time', col_wrap=4, vmax=0.08) # 0.08 for VH

In [ ]:
bg_VV = generate_bg_from_s1(cube,bg_band="VV", fused_by="min")
bg_VH = generate_bg_from_s1(cube,bg_band="VH", fused_by="min")

In [ ]:
print(bg_VV.quantile(q=np.arange(0.6, 0.7, 0.01)).values)
print(bg_VH.quantile(q=np.arange(0.6, 0.7, 0.01)).values)

In [ ]:
bg_VH, binary_bg_VV, binary_bg = generate_landwater_mask(cube, threshold_quantile=0.625)

In [ ]:
(cube.VH * binary_bg).plot.imshow(col='time', col_wrap=4, vmax=0.08)

### Generate BG from S2

In [ ]:
subdir = 'lat_{}_lon_{}'.format(str(lat).replace('.','_'), str(lon).replace('.','_'))
S2_bg_ndwi = os.path.join("/home/jovyan/data/chips", subdir, "bg_ndwi.png")

import skimage.io
bg_ndwi = skimage.io.imread(S2_bg_ndwi)
plt.imshow(bg_ndwi)                  

In [ ]:
len(cube.lat), len(cube.lon)

In [ ]:
from skimage.transform import rescale, resize
bg_ndwi_resize = resize(bg_ndwi, (len(cube.lat), len(cube.lon)), anti_aliasing=True)
print(bg_ndwi_resize.shape)
plt.imshow(bg_ndwi_resize)

In [ ]:
plt.hist(bg_ndwi_resize.reshape(1, -1).squeeze(0))
# np.quantile(bg_ndwi_resize, q=0.5)
threshold_s2 = (bg_ndwi_resize.min() + bg_ndwi_resize.max()) / 3
binary_bg_ndwi = (bg_ndwi_resize < threshold_s2).astype(int)

In [ ]:
plt.imshow(binary_bg_ndwi)

In [ ]:
binary_bg_ndwi = xr.DataArray(binary_bg_ndwi, dims=('lat', 'lon'))
stacked_bg_ndwi = xr.concat([binary_bg_ndwi for i in range(len(cube.time))], dim='time')

In [ ]:
(stacked_bg_ndwi * cube.VH).plot.imshow(col='time', col_wrap=4, vmax=0.08)

### Method here https://github.com/sentinel-hub/custom-scripts/tree/master/sentinel-1/water_surface_roughness_visualization

In [ ]:
cube["WR"] = - np.log(0.05 / (0.018+cube.VV * 1.5)) # turn to minus compare to the original script
cube["WR"].plot.imshow(col='time', col_wrap=4, cmap='Greys')

In [ ]:
water_roughness_cube_bg, bg_water_roughness = preprocess_sentinel_1(cube, band = "WR", fused_by = "min")
bg_water_roughness.plot.imshow(cmap='Greys')

In [ ]:
water_roughness_cube_bg.plot.imshow(col='time', col_wrap=4, cmap='Greys')

In [ ]:
def overlay(top, bottom):    
    res = ((1 - 2 * top) * bottom + 2 * top) * bottom
    return res


def stretch(arr, min_, max_):
    delta = max_ - min_
    offset = - min_ / delta
    return arr/delta + offset

def gamma(arr, val):
    return arr ** (1.0 / val)

mvh = np.sqrt(cube.VH + 0.002)
mvv = np.sqrt(cube.VV + 0.002)
ov = overlay(mvv, mvh)
red = gamma(stretch(mvh, 0.02, 0.1), 1.1)
green = gamma(stretch(ov, 0.0, 0.06), 1.1)
blue = gamma(stretch(mvv, 0.0, 0.32), 1.1)


In [ ]:
cube['R'] = red
cube['G'] = green
cube['B'] = blue


In [ ]:
cube_bands = cube.expand_dims("band")
cube_rgb = xarray.concat([cube_bands.R, cube_bands.G, cube_bands.B], dim="band")


In [ ]:
cube_rgb.plot.imshow(col='time', col_wrap=4, vmin=0, vmax=1, cmap='RdYlBu')

In [ ]:
cube_bands